In [1]:
from inference_trt import inference

In [2]:
inference()

[53.20991897583008, 2.8072762489318848, 0.10085511207580566, 0.05199003219604492, 0.062244415283203125, 0.06456780433654785, 0.07754206657409668, 0.05283808708190918, 0.043929338455200195, 0.050052642822265625, 0.04724717140197754, 0.0450892448425293, 0.04251909255981445, 0.0439000129699707, 0.03811502456665039, 0.03845095634460449, 0.038017988204956055, 0.03814363479614258, 0.040247440338134766, 0.03665900230407715, 0.035363197326660156, 0.0353245735168457, 0.03520798683166504, 0.03498053550720215, 0.034768104553222656, 0.035648345947265625, 0.035407066345214844, 0.03889298439025879, 0.03551769256591797, 0.0364532470703125, 0.03605341911315918, 0.03499436378479004, 0.03550839424133301, 0.03555750846862793, 0.03586459159851074, 0.037386417388916016, 0.03552365303039551, 0.035482168197631836, 0.03655576705932617, 0.035918235778808594, 0.036026716232299805, 0.03667283058166504, 0.03598332405090332, 0.035816192626953125, 0.03566145896911621, 0.03573870658874512, 0.03691983222961426, 0.036

In [11]:
import torch
import torch.onnx
from mobilenet_v2_tsm import MobileNetV2
import io
import onnx

torch_module = MobileNetV2(n_class=27)
torch_module.load_state_dict(torch.load("mobilenetv2_jester_online.pth.tar"))

<All keys matched successfully>

In [6]:
torch_module.eval()
torch_inputs = (torch.rand(1, 3, 224, 224),
                torch.zeros([1, 3, 56, 56]),
                torch.zeros([1, 4, 28, 28]),
                torch.zeros([1, 4, 28, 28]),
                torch.zeros([1, 8, 14, 14]),
                torch.zeros([1, 8, 14, 14]),
                torch.zeros([1, 8, 14, 14]),
                torch.zeros([1, 12, 14, 14]),
                torch.zeros([1, 12, 14, 14]),
                torch.zeros([1, 20, 7, 7]),
                torch.zeros([1, 20, 7, 7]))
input_names = []
input_shapes = {}
with torch.no_grad():
    for index, torch_input in enumerate(torch_inputs):
        name = "i" + str(index)
        input_names.append(name)
        input_shapes[name] = torch_input.shape
    buffer = io.BytesIO()
    torch.onnx.export(torch_module, torch_inputs, buffer, input_names=input_names, output_names=["o" + str(i) for i in range(len(torch_inputs))], opset_version=10)
    outs = torch_module(*torch_inputs)
    buffer.seek(0, 0)
    onnx_model = onnx.load_model(buffer)
    from onnxsim import simplify
    onnx_model, success = simplify(onnx_model)  # this simplifier removes conversion bugs.
    assert success
    onnx.save(onnx_model, './mobilev2.onnx')

/usr/local/lib/python3.6/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /media/nvidia/NVME/pytorch/pytorch-v1.9.0/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


In [13]:
import tensorrt as trt
from torch2trt import torch2trt

In [18]:
torch_module = torch_module.eval().cuda()
torch_inputs = (
                torch.zeros([1, 3, 224, 224], device='cuda'),
                torch.zeros([1, 3, 56, 56], device='cuda'),
                torch.zeros([1, 4, 28, 28], device='cuda'),
                torch.zeros([1, 4, 28, 28], device='cuda'),
                torch.zeros([1, 8, 14, 14], device='cuda'),
                torch.zeros([1, 8, 14, 14], device='cuda'),
                torch.zeros([1, 8, 14, 14], device='cuda'),
                torch.zeros([1, 12, 14, 14], device='cuda'),
                torch.zeros([1, 12, 14, 14], device='cuda'),
                torch.zeros([1, 20, 7, 7], device='cuda'),
                torch.zeros([1, 20, 7, 7], device='cuda'))
model_trt = torch2trt(torch_module, [*torch_inputs])

In [19]:
torch.save(model_trt.state_dict(), 'mobilenetV2_trt.pth')

In [20]:
y = torch_module(*torch_inputs)
y_trt = model_trt(*torch_inputs)

TypeError: unsupported operand type(s) for -: 'tuple' and 'tuple'

In [24]:
y[0]

tensor([[ 1.1721,  0.0217,  1.7744,  0.3946,  0.6725, -0.0780,  0.3258, -0.2726,
         -0.1468, -0.0497,  0.4375, -1.6775, -1.1119,  0.5433,  0.4191,  0.3638,
         -1.6851, -1.2728,  0.5660, -0.4239,  0.2610, -0.0367, -0.1212, -0.0310,
          0.4124, -0.4000, -0.0573]], device='cuda:0', grad_fn=<AddmmBackward>)

In [23]:
y_trt[0]

tensor([[ 1.1721,  0.0217,  1.7744,  0.3946,  0.6725, -0.0780,  0.3258, -0.2726,
         -0.1468, -0.0497,  0.4375, -1.6775, -1.1119,  0.5433,  0.4191,  0.3638,
         -1.6851, -1.2728,  0.5660, -0.4239,  0.2610, -0.0367, -0.1212, -0.0310,
          0.4124, -0.4000, -0.0573]], device='cuda:0')